In [2]:
import os
os.chdir('/lustre/scratch/kiviaho/hillock_club_senescence')

import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

from scipy.stats import pearsonr


Matplotlib created a temporary config/cache directory at /tmp/matplotlib-jaqffhpj because the default path (/run/cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:

# Without excluding any clusters
adata_ref_path = '../prostate_spatial/aggregate_sc_data_with_broad_annotation_20230712.h5ad'
adata_ref = sc.read_h5ad(adata_ref_path)

adata_ref.obs['phenotype'] = adata_ref.obs['phenotype'].cat.reorder_categories(['normal','PCa','CRPC'])


In [ ]:
# Score the NMF-derived epithelial modules
gene_modules = pd.read_excel('./gene_modules.xlsx')
for mod in gene_modules:
    lst = gene_modules[mod].dropna().tolist()
    sc.tl.score_genes(adata_ref, gene_list=lst, use_raw=False, score_name=mod, ctrl_size=len(lst), random_state=4368456)


## Plot marker gene expression in a dotplot of the whole aggregate dataset

In [ ]:
def keep_specified_markers(marker_list, keep_vals):
    new_marker_list = {}
    for k, v in marker_list.items():
        new_marker_list[k] = [val for val in v if val in keep_vals]
    return new_marker_list


refined_markers = {
    'Epithelial': ['EPCAM', 'KRT18', 'KRT8','KLK3','AR', 'MSMB','KRT5', 'KRT15', 'TP63', 'KRT7', 'KRT19', 'KRT4'],
    'Endothelial': ['VWF', 'SELE', 'FLT1', 'ENG'],
    'Fibroblast': ['LUM', 'DCN', 'IGF1', 'APOD', 'STC1', 'FBLN1', 'COL1A2', 'C7', 'IGFBP5', 'ACTA2'],
    'CAFs':['FAP','NDRG2', 'TSPAN1', 'PTN', 'APOE', 'OR51E2', 'P4HB', 'STEAP1','ABCC4'], # From here https://www.nature.com/articles/s41598-023-36125-0
    'metCAFs':['ITGA11','THBS1','FN1','EMP1','ITGA2','FYN','SPP1','EMP2'], # from here https://translational-medicine.biomedcentral.com/articles/10.1186/s12967-024-05413-2
    'iCAFs':['IL13RA2', 'GDF7', 'IL33', 'CXCL1', 'TNFRSF19', 'CXCL6', 'LIFR', 'CXCL5', 'IL7','TSLP','TNFSF15'],
    'SMC': ['RGS5', 'ACTA2', 'TAGLN', 'BGN', 'MYL9', 'MYLK','CALD1',],
    'Mast': ['KIT', 'TPSB2', 'TPSAB1', 'CPA3', 'VWA5A', 'IL1RL1', 'CTSG', 'SLC18A2', 'ACSL4', 'MS4A2', 'GATA2'],
    'T cell': ['CD3D', 'CD3E', 'CD3G', 'CD8A', 'CD8B', 'IL7R'],
    'B cell': ['CD79A', 'MS4A1', 'CD79B', 'IGHM', 'CD83'],
    'Myeloid': ['CD4', 'C1QA', 'C1QB', 'C1QC', 'CD68', 'LYZ', 'IL1B', 'S100A9', 'S100A8','CXCL8','FCGR3A', 'CSF1R',],
    'Neuronal': ['PLP1', 'MPZ', 'MT1H'],
    'Dendritic': ['IRF7', 'IRF4', 'FCER1A', 'CD1C'],
    'Plasma': ['IGJ', 'MZB1']
}



### Plot a dotplot with gene markers

In [ ]:

sns.set_theme(style='white',font_scale=0.8)
    
markers = keep_specified_markers(refined_markers, adata_ref.var_names.tolist())
sc.tl.dendrogram(adata_ref,groupby='VI_clusters',use_rep='X_scVI')
fig,ax = plt.subplots(figsize=(8,12))
sc.pl.dotplot(adata_ref, markers, groupby='VI_clusters', dendrogram=True, log= False,
            swap_axes = True, vmax=4,
            ax=ax,show=False,
            )
plt.tight_layout()
plt.savefig(f'./plots/scs_dataset_dotplots/aggregate_data_celltype_marker_dotplot_with_CAFs.png',dpi=120)
#plt.savefig(f'./plots/scs_dataset_dotplots/{dataset}_celltype_marker_dotplot_with_CAFs.pdf')



### Plot an UMAP with cell annotations

In [ ]:
### Plot UMAPs separately for each dataset for cell type identification
sns.set_theme(style='white',font_scale=1)

fig,ax = plt.subplots(figsize=(5,5))
sc.pl.umap(adata_ref, color='refined_celltypes', ax=ax,show=False,legend_loc="on data")
plt.tight_layout()
plt.savefig(f'./plots/scs_dataset_umaps/aggregate_data_celltype_annotation_umap.png',dpi=120)
#plt.savefig(f'./plots/scs_dataset_umaps/{dataset}_celltype_cluster_umap.pdf')



## Investigate IL1A & IL1B expression in epithelial clusters

In [ ]:

# Sample-specific clusters exclusion
adata_ref_path = '../prostate_spatial/single_cell_reference_with_nmf_derived_annotations_20230908.h5ad'
adata_ref = sc.read_h5ad(adata_ref_path)

adata_ref = adata_ref[adata_ref.obs['refined_celltypes'] == 'Epithelial']

In [ ]:
sc.tl.rank_genes_groups(adata_ref,
    groupby='final_annotation',
    use_raw = True, # Raw layer contains the normalized counts
    groups = 'all',
    method='wilcoxon'
)


df_list = []
for clust in adata_ref.obs['final_annotation'].cat.categories:
    df = sc.get.rank_genes_groups_df(adata_ref,group = clust,pval_cutoff = 0.05, log2fc_min = 1)
    df['celltype'] = f'{clust}'
    df_list.append(df)



deg_cluster_markers = pd.concat(df_list, axis=0)

deg_cluster_markers  = deg_cluster_markers.set_index('names')

rename_dict = {
    'interferon signaling epithelium': 'club_interferon response epithelium',
    'mesenchymal epithelium':'sensescent epithelium',
    'cycling epithelium' : 'cycling epithelium 2',
    'FOSL1 tumor epithelium' : 'FOSL1 related epithelium'

}

deg_cluster_markers = deg_cluster_markers.replace(rename_dict)

In [ ]:

genes_of_interest = ['TNF','IL1A','IL1B','IL6','NFKB1']
deg_cluster_markers.loc[genes_of_interest]


### Compare according to the phenotype

In [ ]:
sc.tl.rank_genes_groups(adata_ref,
    groupby='phenotype',
    use_raw = True, # Raw layer contains the normalized counts
    groups = 'all',
    method='wilcoxon'
)


df_list = []
for clust in adata_ref.obs['phenotype'].cat.categories:
    df = sc.get.rank_genes_groups_df(adata_ref,group = clust,pval_cutoff = 0.05, log2fc_min = 1)
    df['celltype'] = f'{clust}'
    df_list.append(df)



deg_cluster_markers_phenotype = pd.concat(df_list, axis=0)

deg_cluster_markers_phenotype  = deg_cluster_markers_phenotype.set_index('names')

In [ ]:
deg_cluster_markers_phenotype.loc[[c for c in genes_of_interest if c in deg_cluster_markers_phenotype.index.tolist()]]

In [ ]:
sc.pl.violin(adata_ref,
    groupby='phenotype',
    keys=['IL1A','IL1B'],
    use_raw=True,
    show=False,
    save=True
)


In [ ]:
(adata_ref[adata_ref.obs['phenotype'] == 'CRPC'][:,['IL1A','IL1B']].X.todense() != 0).sum(axis=0) / len(adata_ref)

## Investigate the IL1B+ TAMs score in monocytes

In [ ]:
# Subset
myeloid_cells = adata_ref[adata_ref.obs['refined_celltypes'] == 'Myeloid']


### Define gene signatures and for scoring

In [8]:
## Load the list of markers

# Get the Caronni et al. 2023 PDAC TAM subset gene signature
pdac_tam_markers = pd.read_excel('caronni_et_al_2023_pdac_tam_markers.xlsx')
pdac_tam_markers = pdac_tam_markers[['Gene','Cell_population']].pivot(values='Gene',columns='Cell_population')

for col in pdac_tam_markers:
    pdac_tam_markers[col] = pdac_tam_markers[col].dropna().reset_index(drop=True)

pdac_tam_markers = pdac_tam_markers.dropna(axis=0,how='all')

# Create a dictionary with the lists inside
data_dict = {
    'PDAC Macrophages': ['APOC1','C1QB','APOE','C1QC','C1QA','SPP1','CCL3L1','CCL3','TREM2','MSR1','GPNMB','MS4A7','SLCO2B1','FCGR3A','FCGR2A','MS4A4A','SLC16A10','CD14','CSF1R','RNASE1','CD68','SDS','VSIG4','MRC1','CYBB'],
    'PDAC Neutrophils': ['S100A8','CXCL8','FCGR3B','IL1R2','S100A12','PROK2','NAMPT','CSF3R','CMTM2','S100A9','BCL2A1','AC245128.3','ADGRG3','IFITM2','AL034397.3','SAMSN1','IVNS1ABP','AQP9','PTGS2','GCA','BASP1','G0S2','FPR1','PLEK','PHACTR1','FPR2','LITAF','ANP32A','ACSL1','MNDA','RIPOR2','NCF1','ALOX5AP','CPD','SMIM25','TREM1','IRAK3','SRGN','RNF149','NABP1','LST1','NSMAF','LCP1','MME','SLA','SELL','CXCR2','LCP2','HCAR2','C5AR1'],
    'PDAC Monocytes': ['S100A8','FCN1','S100A9','S100A12','SERPINB2','EREG','CD300E','AC245128.3','APOBEC3A','NLRP3','SLC11A1','AQP9','THBS1','FPR1','MCEMP1','C5AR1','IL1B','SMIM25','ATP2B1-AS1','CSF3R','SLC43A2','CFP','IL1R2','THBD','FPR2'],
    'PDAC Dendritic cells': ['CD1C','CLEC10A','FCER1A','CD1E','HLA-DQB1','HLA-DQA2','HLA-DPB1','HLA-DQA1','HLA-DPA1','S100B','CPVL','HLA-DRB5','HLA-DRA','PKIB','JAML','CSF2RA','HLA-DRB1','CD86','MS4A6A','GPR183','RNASE6','FCGR2B','CD74','IGSF6','HLA-DMB'],
    'PMN-MDSC activity': ['RPL21','XAF1','CLEC5A','CAMP','CD63','LTF','ANXA3','S100A8','OAS2','ALOX5','IFIT1','CD177','S100A9','CEBPE','LCN2','ANXA1','CTSG','MPO','GSTM1','MEGF9','PYGL','PRTN3','YTHDF3','EMILIN1','ELANE','CHI3L1','LTA4H','VILL','HMGN2','C3','MS4A3','ITGAM','CLEC12A','CST7','HSD11B1','C5AR1','F13A1','ADPGK','HP','IDH1','DHRS7','PGLYRP1','MMP9','ALDH3B1','IGSF6','CYBB','AP3S1','NKG7','RAB31','MSRA','OAS3','SLPI','SYNE1','MECR','PRDX5','MCFD2','RAB3D','PILRA','SCP2','ALAS1','CPNE3','NCF1','MAPKAPK3','ACOT7','CD9','IFITM3','MMP8','PLAC8','NAMPT','ZMPSTE24','ARHGDIB','LGALS1','PADI4','ENO1','DSTN','IFI27L1','MYO1F','ETHE1','CEACAM1','SH3BGRL3','GSN'],
    'Hirz MDSCs': ['CEBPB','IL10','NOS2','RORC','S100A8','SOCS1','SOCS3','TGFB1','IL6','CSF2','CSF1','FLT3LG','ARG1','PTGS2','VEGFA','TNF','S100A9','CYBB','NCF1','NCF4','CSF3R','CXCL8','MNDA','LYZ','NCF2','SELL','ICAM1','CD63','CD274','OLR1'],
    'Calcinotto MDSCs': ['CCL15','CCL18','CCL19','CCL20','CCL23','CCL24','CCL27','CCL3','CCL3L3','CCL4','CCL4L2','CXCL10','CXCL12','CXCL13','CXCL16','CXCL6','CXCR1','CXCR2','CXCR3','CXCR4','CXCR5','CXCR6','IFNA1','IFNA10','IFNA13','IFNA14','IFNA16','IFNA17','IFNA2','IFNA21','IFNA4','IFNA5','IFNA6','IFNA7','IFNA8','IFNAR2','IFNB1','IFNG','IL10','IL10RA','IL11RA','IL12A','IL12B','IL12RB1','IL12RB2','IL13','IL17B','IL17F','IL17RA','IL18RAP','IL19','IL1A','IL1B','IL1R2','IL1RAP','IL1RL1','IL1RN','IL21','IL22','IL22RA2','IL23A','IL23R','IL24','IL27','IL2RA','IL2RB','IL2RG','IL4','IL4R','IL5','IL5RA','IL6','IL7','IL7R','IL9','LTA','LTB','LTF','MPO','TGFB2','TNFRSF11B','TNFRSF13C','TNFRSF14','TNFRSF17','TNFRSF18','TNFRSF4','TNFRSF8','TNFSF10','TNFSF11','TNFSF14','TNFSF4','TNFSF8']
}

# Find the maximum length of the lists
max_length = len(pdac_tam_markers)

# Lengthen each list with np.nan to match the maximum length
for key in data_dict:
    data_dict[key] += [np.nan] * (max_length - len(data_dict[key]))

# Create a DataFrame from the dictionary
new_df = pd.DataFrame(data_dict)

# Assuming pdac_tam_markers is your existing DataFrame
# Concatenate the new DataFrame with pdac_tam_markers
pdac_tam_markers = pd.concat([pdac_tam_markers, new_df], axis=1)

all_genes = [c for c in list(np.array(pdac_tam_markers).ravel()) if not pd.isnull(c)]

# Score the NMF-derived epithelial modules
for mod in pdac_tam_markers:
    lst = pdac_tam_markers[mod].dropna().tolist()
    sc.tl.score_genes(myeloid_cells, gene_list=lst, use_raw=False, score_name=mod, ctrl_size=len(lst), random_state=4368456)

all_genes = [c for c in list(np.array(pdac_tam_markers).ravel()) if not pd.isnull(c)]


### Reclustering based on the scVI embedding

In [ ]:
# Recluster the myeloid cells based on their scVI components
sc.pp.neighbors(myeloid_cells, use_rep="X_scVI",random_state=2154624)
sc.tl.leiden(myeloid_cells, key_added="VI_clusters",resolution=0.25) # 0.36
sc.tl.umap(myeloid_cells,random_state=2154624)
#myeloid_cells.uns.pop('VI_clusters_colors')

### Differentially expressed genes between the clusters

In [ ]:
sc.tl.rank_genes_groups(myeloid_cells,
    groupby='VI_clusters',
    groups = 'all',
    method='wilcoxon'
)


df_list = []
for clust in myeloid_cells.obs['VI_clusters'].cat.categories:
    df = sc.get.rank_genes_groups_df(myeloid_cells,group = clust,pval_cutoff = 0.05, log2fc_min = 1)
    df['cluster_res0.25'] = f'cluster{clust}'
    df_list.append(df)



deg_cluster_markers = pd.concat(df_list, axis=0)

# Keep only genes that appear once
#deg_cluster_markers = deg_cluster_markers[deg_cluster_markers['names'].isin((deg_cluster_markers['names'].value_counts()[deg_cluster_markers['names'].value_counts() == 1]).index.tolist())].reset_index(drop=True)

deg_cluster_markers.to_excel('./prostate_monocyte_cluster_markers_from_aggregate_dataset.xlsx')


### Fraction of IL1A+/IL1B+ myeloid cells in normal/PCa/CRPC

In [ ]:
df = adata_ref.obs[['phenotype','refined_celltypes']].copy()

# Create a binary annotation column

for phtype in df['phenotype'].cat.categories:
    df.loc[:,phtype] = 'no'
    df.loc[df[df['phenotype'] == phtype].index,phtype] = 'yes'
    df[phtype] = pd.Categorical(df[phtype],categories = ['yes','no'])

for cl in myeloid_cells.obs['VI_clusters'].cat.categories:
    df.loc[:,'cluster ' + cl] = 'no'
    df.loc[myeloid_cells.obs[myeloid_cells.obs['VI_clusters']==cl].index,'cluster ' + cl] = 'yes'
    df['cluster ' + cl] = pd.Categorical(df['cluster ' + cl],categories = ['yes','no'])


df_subset = df[df['refined_celltypes'] == 'Myeloid'].copy()

In [ ]:
from scipy.stats import fisher_exact

phenotypes = df['phenotype'].cat.categories.tolist()
clusters = myeloid_cells.obs['VI_clusters'].cat.categories.tolist()

oddsratio_df = pd.DataFrame(index = phenotypes, columns = myeloid_cells.obs['VI_clusters'].cat.categories,dtype=np.float64)
pval_df = pd.DataFrame(index = phenotypes, columns = myeloid_cells.obs['VI_clusters'].cat.categories,dtype=np.float64)

for phtype in phenotypes:
    for cl in clusters:
        crosstab = pd.crosstab(df_subset[phtype],df_subset['cluster ' + cl])
        stat, pval = fisher_exact(crosstab)
        oddsratio_df.loc[phtype,cl] = stat
        pval_df.loc[phtype,cl] = pval

log_oddsratio_df = np.log2(oddsratio_df)

annot = pd.DataFrame('',index = pval_df.index, columns = pval_df.columns)
annot[pval_df < 0.05] = '*'

In [ ]:
# Plot the heatmap
plt.figure(figsize=(6, 3))
sns.heatmap(log_oddsratio_df, annot=annot,
    linewidths = 1,
    linecolor='white',
    cmap='bwr',
    fmt='s',
    square=True,
    annot_kws={'fontsize':20,'ha':'center','va':'center'}
    )

plt.xlabel('Myeloid cell cluster')
plt.ylabel('scRNA-seq sample')

plt.tight_layout()
plt.savefig('./tmp/plot.png',dpi=200)

### UMAP of VI clusters

In [ ]:
# Plot the UMAP

#dat = myeloid_cells[myeloid_cells.obs['dataset'] != 'hirz_2023']
dat = myeloid_cells

sns.set_theme(style='white')


# Flatten the axs array for easier iteration
fig, ax = plt.subplots(figsize=(5,3))

sc.pl.umap(
    dat,
    color='VI_clusters',
    #color='dataset',
    ax=ax
)

plt.tight_layout()
plt.savefig('./tmp/plot.png')



In [ ]:

# Plot the six different scores on the UMAP

sns.set_theme(style='whitegrid')

set_names = pdac_tam_markers.columns.tolist()

fig, axs = plt.subplots(6, 2, figsize=(8, 18))

# Flatten the axs array for easier iteration
axs = axs.flatten()

for idx, set_name in enumerate(set_names):
    ax = axs[idx]
    sc.pl.umap(
    myeloid_cells,
    color=set_name,
    cmap='Reds',
    vmin ='p1',
    vmax ='p99',
    ax=ax
)


plt.tight_layout()
plt.savefig('./tmp/plot.png')



In [ ]:

sns.set_theme(style='whitegrid')

set_names = pdac_tam_markers.columns.tolist()

fig, axs = plt.subplots(6, 2, figsize=(8, 18))

# Flatten the axs array for easier iteration
axs = axs.flatten()

for idx, set_name in enumerate(set_names):
    score_q = myeloid_cells.obs[set_name].quantile(0.5)
    ax = axs[idx]
    sc.pl.violin(
        myeloid_cells,
        groupby='VI_clusters',
        keys=set_name,
        stripplot=False,
        ax=ax
    )
    ax.axhline(y = score_q, linestyle='--',c='k')

plt.tight_layout()
plt.savefig('./tmp/plot.png')



In [ ]:
# Plot individual genes expression in violin
# List of genes
genes = [ "HLA-DRA", "ITGAX", "IL3RA"] # "CD1C", "CD1B","CD1A","FCER1A",

# Set the theme for the plot
sns.set_theme(style='whitegrid')

# Create a figure with 5 subplots, one for each gene
fig, axs = plt.subplots(len(genes), 1, figsize=(4, int(3*len(genes))))

# Flatten the axs array for easier iteration
axs = axs.flatten()

# Plot violin plots for each gene
for i, gene in enumerate(genes):
    ax = axs[i]
    sc.pl.violin(
    myeloid_cells,
    groupby='VI_clusters',
    keys=gene,
    stripplot=False,
    use_raw=False,
    ax=ax,
    show=False
)
 

plt.tight_layout()
plt.savefig('./tmp/plot.png')


In [ ]:
from scipy.stats import fisher_exact

PMN_MDSC_signature = ['STAT1','STAT3','STAT6','NFKB1','IRF1','S100A9','S100A8','ANXA1','LYZ2',
                     'CXCL1','CXCL2','CXCR1','CXCR2','CXCL8','LILRA3','TREM1','PTGS2','ARG1','ARG2'
                     'TGFB1','VEGF','IL6','CSF1','IL1B','WFDC17','IL4R','OLR1','CD84']

club_region_signature = ['SLPI','MMP7','ELF3','TNFAIP2','MUC1','CLDN4','PIGR','CP','GPRC5A','TNFRSF21','KRT7','RHOV','NCOA7','GABRP','LXN','ITGB6','S100P','KRT19','LCN2','CFB','CLDN1','ASS1','ATP1B1','CDH3','KRT17','LTF','ANXA2','KLF5','S100A2','VMP1','CXCL17','WFDC2','RARRES1','CEACAM1','DUSP4','TNFSF15','NPC2','SCUBE2','MPZL2','PI3','CXCL1','CXCL16','SOD2','OLFM4','CXCL2','PRSS22','CFTR','ST6GAL1','MET','ATF3','CRABP2','SCNN1A','BCL3','SERPINA1','SCGB3A1','PTGES','MUC4','S100A9','PTGS2','LAMB3','EPHA2','PIM3','UBD','ALS2CL','TJP3','TMEM63A','PDZK1IP1','C19orf33','MYH14','MACC1','ITGA3','ID1','LAMC2','PTPRK','SOX9','ACSL5','TMEM45B','BACE2','NTN4','GRB7','CXCL6','SAA1','RNF145','CXCL3','SEMA4B','SLC5A1','TNFSF10','TM4SF1','PLPP2','BIRC3','SDC4','AQP3','TACSTD2','KRT23','PLEKHS1','BTG3','AHR','VNN2','ACHE','KCNQ1','GABRE','ERP27','DRAM1','ITGB4','ADGRF1','S100A11','INAVA','S100A14','CEBPB','CLDN7','AC254629.1','KLK10','STEAP3','AC020916.1','LIMK2','SLC15A2','KRT80','AL049839.2','PPL','DHRS3','RARRES3','DDR1','TRIM29','ZC3H12A','RNF207','DKK1','KRT8','GGT6','ERBB2','PLAU','KRT5','SHB','MUC3A','KRT15','RAB11FIP1','PADI2','SLC26A4','GSTA1','EDN1','ABCC3','PHLDA2','TRAF4','AGR2','OCLN','SDC1','DUSP23','MYO1B','SHROOM3','ODF3B','RUNX1','ITGB8','DEFB1','ARHGAP26','BAG1','CXCL8','EPS8L2','TMC5','JUP','PDE4B','GADD45A','NCEH1','KRT4','SUSD6','VTCN1','KIAA1671','BPIFB1','GLUL','PROM1','PLEKHA6','TNS4','PERP','CXADR','ZFP36L1','TGM2','NGFR','DUSP5','FAM3D','CCL2','FMO5','SBNO2','CYP4F12','ATP1A1','AGRN','TFCP2L1','S100A16','GRHL1','EPS8L1','RGCC','TNFRSF12A','PAPSS1','C12orf49','LRG1','ACSF2','AC007906.2','CRACR2B','ETS2','HBEGF','EFNA1','RIPK4','CHPT1','ITPKC','RAB15','LOXL4','EGLN3','PNPLA2','CCL20','LIMA1','CAPN8','PTPRU','ZSWIM4','CLDN10','PPP1R13L','KCNK1','TYMP','SERINC2','NFKBIA','FLRT3','MISP','GPX2','CTSH','ST14','HES1','MECOM','BICDL2','FOXQ1','BHLHE40','IGFBP3','SAT1','AUTS2','DCDC2','TPBG','SLC5A6','RARG','ETV6','EXPH5','TCIM','MYOF','NEURL3','NEDD9','CD82','PROM2','F3','SORL1','SPINT1','EZR','GATA3','MPZL3','CHI3L2','IRF1','CREB3L1','LINC02009','C5orf46','NECTIN4','SIRPA','ERO1A','GSAP','GDF15','HSBP1L1','NPAS2','ERRFI1','ARRDC2','PLD2','BMP2','C1RL','PDXK','B3GNT3','RBM7','TMEM45A','CYP24A1','FGFBP1','AL591895.1','SPP1','PTP4A1','SELENBP1','WNT7B','FRAS1','PATJ','HK2','EDN2','MICALL2','RIN2','SPIB','KCNN4','F11R','SLC44A4','FCGBP','CX3CL1','KRTCAP3','DDIT4','NFKBIZ','CHL1','ZFP36L2','INPP5J','SDC2','IVNS1ABP','GNA15','IFNGR1','NFIB','SLC11A2','IFNGR2','SYNE2','MIDN','TF','MARCKS','CLEC3A','TFRC','KIAA1217','FHL2','SH2D3A','ARRDC3','TFF1','RAB20','GJB3','CTSB','CST6','GNPTAB','ARHGEF28','CDCP1','SERPINB1','DTX4','SLC35C1','ENC1','TNS3','SFN','SLC4A4','ESPN','DLG5','ADGRG6','SAA2','KLF6','MUC20','LLGL2','DEPP1','CTNND1','TP53I3','WEE1','C2orf54','MGAT4B','SLC34A2','PSME2','SMURF1','RELB','SLC43A2','ZMIZ2','ITGA2','TMEM178A','FUT2','PARP4','NFKB2','CXCL5','SPTSSB','ABHD11','RALGDS','IER5','EVA1C','TFAP2A','KIAA1522','PI15','KRT13','CNFN','SH3BP4','LGALS3','PABPC1L','HLA-F','MAST4','FAT1','HS6ST1','PCDH8','TAPBP','B3GNT5','GALNT18','IFFO2','CLCF1','CHAC1','PISD','MMP10','FAM110C','HS3ST1','TUBA1C','KRT16','FMO2','CHI3L1','CLDN3','KRT18','MAL2','TFAP2C','SLC40A1','IER3','CEBPD','LYZ','SUSD4','TSPAN12','ITPR3','RASD1','HLA-DMA','FBP1','THSD4','FAAH2','PPP1R1B','ROBO1','UNC93B1','MAP3K1','PLCH2','MYO5C','MAFF','CELSR1','BSPRY','KLK11','IER2','LSR','TMEM123','TRAM1','USP54','HLA-DQB1','TRIM47','OSBPL10','DAB2IP','GPR87','STAC2','ADK','TBC1D2','WWC1','ADGRG1','IRX3','DMKN','TM7SF3','TRIM2','NFIL3','CFI','TTC7A','B4GALT1','TRIB1','PTPN3','ERBB3','ICAM1','RAB25','PRR15L','TAP1','PDE8B','TRPV4','TNFRSF10B','SECTM1','AMOT','TSPAN15','PLSCR1','SLC4A11','CITED4','NNMT','PPFIBP2','VAMP8','CLMN','OCIAD2']

sig = PMN_MDSC_signature
sig2 = deg_cluster_markers[deg_cluster_markers['cluster_res0.25'] == 'cluster2']['names'].tolist()
#sig2 = PMN_MDSC_signature

overlap = list(set(sig2).intersection(set(sig)))

print(overlap)

a = len(overlap)
b = len(sig) - a
c = len(sig2) - a
d = len(myeloid_cells) - len(sig2)

arr = np.array([[a,b],[c,d]])
stat, pval = fisher_exact(arr)

print(f'p = {pval:.2e}')

### Celltype proportions in samples

In [ ]:
adata_ref.obs['phenotype'].value_counts()

# PCa       183184 0.558878
# normal    106597 0.325218
# CRPC       37990 0.115904

adata_ref[adata_ref.obs['refined_celltypes'] == 'Myeloid'].obs['phenotype'].value_counts()

# PCa       15881 0.631652
# normal     7625 0.303277
# CRPC       1636 0.065070

In [ ]:
cl = '3'
myeloid_cells[myeloid_cells.obs['VI_clusters'] == cl].obs['phenotype'].value_counts() / len(myeloid_cells[myeloid_cells.obs['VI_clusters'] == cl])

### Reclustering based on raw expression

In [ ]:
# Recluster the myeloid cells based on their gene expression of marker genes

# Set the highly variable genes as the PDAC TAMs marker genes
#myeloid_cells.var['highly_variable'] = myeloid_cells.var_names.isin(all_genes)

sc.pp.highly_variable_genes(myeloid_cells, min_mean=0.0125, max_mean=3, min_disp=0.5)
sc.tl.pca(myeloid_cells, svd_solver="arpack")
sc.pp.neighbors(myeloid_cells,random_state=2154624)
sc.tl.leiden(myeloid_cells, key_added="expr_clusters",resolution=0.36)
sc.tl.umap(myeloid_cells,random_state=2154624)



In [ ]:

sns.set_theme(style='whitegrid')

set_names = pdac_tam_markers.columns.tolist()

fig, axs = plt.subplots(3, 2, figsize=(8, 9))

# Flatten the axs array for easier iteration
axs = axs.flatten()

for idx, set_name in enumerate(set_names):
    score_q = myeloid_cells.obs[set_name].quantile(0.5)
    ax = axs[idx]
    sc.pl.violin(
        myeloid_cells,
        groupby='expr_clusters',
        keys=set_name,
        stripplot=False,
        ax=ax
    )
    ax.axhline(y = score_q, linestyle='--',c='k')

plt.tight_layout()
plt.savefig('./tmp/plot.png')



In [ ]:
sc.tl.rank_genes_groups(myeloid_cells,
    groupby='expr_clusters',
    groups = 'all',
    method='wilcoxon'
)

df_list = []
for clust in myeloid_cells.obs['expr_clusters'].cat.categories:
    df = sc.get.rank_genes_groups_df(myeloid_cells,group = clust,pval_cutoff = 0.05, log2fc_min = 1)
    df['cluster_res0.36'] = clust
    df_list.append(df)

deg_cluster_markers = pd.concat(df_list, axis=0)

#deg_cluster_markers.to_excel('./prostate_monocyte_cluster_markers.xlsx')

In [ ]:
myeloid_cells[myeloid_cells.obs['expr_clusters'] == '1'].obs['phenotype'].value_counts()

In [ ]:
to_drop = [g for g in all_genes if g not in myeloid_cells.var_names]

sns.set_theme(style='white',font_scale=1)
sc.tl.dendrogram(myeloid_cells,groupby='VI_clusters',use_rep='X_scVI')
fig,ax = plt.subplots(figsize=(40,6))
sc.pl.dotplot(myeloid_cells, 
              [g for g in all_genes if g not in to_drop],
              groupby='expr_clusters',
              dendrogram=True,
              log= False,
              swap_axes = False, vmax=4,
              ax=ax,show=False,
              )
plt.savefig('./tmp/plot.png')


## Investigate T1RS and IL1B gene set scores in the single cell data

In [ ]:
# Plot the module scores on UMAPs
ctype = 'all'
sns.set_theme(style='whitegrid',font_scale=0.8)

for set_name in gene_modules:
    fig, ax = plt.subplots(figsize=(5,5))
    sc.pl.umap(adata_ref,
        color=set_name,
        vmin=0,
        vmax='p99.9',
        use_raw=False,
        cmap='Reds',
        show=False,
        ax=ax,
        frameon=False
        )
    plt.tight_layout()
    name = set_name.replace(' ','_')
    plt.savefig('./plots/{}_expression_umap_{}.pdf'.format(name,ctype))
    plt.savefig('./plots/{}_expression_umap_{}.png'.format(name,ctype))


In [ ]:

for set_name in gene_modules.columns:
    score_q = adata_ref.obs[set_name].quantile(0.95)
    fig, ax = plt.subplots(figsize=(6,4))
    sc.pl.violin(
        adata_ref,
        groupby='refined_celltypes',
        keys= set_name,
        ax=ax
    )
    ax.axhline(y = score_q, linestyle='--',c='k')
    plt.tight_layout()
    name = set_name.replace(' ','_')
    plt.savefig('./plots/{}_expression_violin_{}.pdf'.format(name,ctype))
    plt.savefig('./plots/{}_expression_violin_{}.png'.format(name,ctype))



In [ ]:
q_threshold = 0.99
# Draw the threhold accoring to a cell subset
ctype = 'Myeloid'
set_name = 'PDAC IL1B+ Macrophages'

score_q = adata_ref[adata_ref.obs['refined_celltypes'] == ctype].obs[set_name].quantile(q_threshold)
fig, ax = plt.subplots(figsize=(6,4))
sc.pl.violin(
    adata_ref,
    groupby='refined_celltypes',
    keys= set_name,
    ax=ax
)
ax.axhline(y = score_q, linestyle='--',c='k')
plt.tight_layout()
name = set_name.replace(' ','_')
plt.savefig('./plots/{}_expression_violin_{}.pdf'.format(name,ctype))
plt.savefig('./plots/{}_expression_violin_{}.png'.format(name,ctype))


# Draw the threhold accorind to a cell subset

ctype = 'Epithelial'
set_name = 'PDAC T1RS'

score_q = adata_ref[adata_ref.obs['refined_celltypes'] == ctype].obs[set_name].quantile(q_threshold)
fig, ax = plt.subplots(figsize=(6,4))
sc.pl.violin(
    adata_ref,
    groupby='refined_celltypes',
    keys= set_name,
    ax=ax
)
ax.axhline(y = score_q, linestyle='--',c='k')
plt.tight_layout()
name = set_name.replace(' ','_')
plt.savefig('./plots/{}_expression_violin_{}.pdf'.format(name,ctype))
plt.savefig('./plots/{}_expression_violin_{}.png'.format(name,ctype))



### Set the top 1% scoring cells and define their DEGs 


In [ ]:
q_threshold = 0.99
#ctype = 'Epithelial' #'Myeloid'
#set_name = 'PDAC T1RS' #'PDAC IL1B+ Macrophages'

ctype = 'Myeloid'
set_name = 'PDAC IL1B+ Macrophages'

subset = adata_ref[adata_ref.obs['refined_celltypes'] == ctype].copy()
top_1_pct_cells = subset[subset.obs[set_name] >= subset.obs[set_name].quantile(q_threshold)].obs_names

subset.obs[f'{set_name} identity'] = 'no'
subset.obs.loc[top_1_pct_cells,f'{set_name} identity'] = 'yes'

In [ ]:
ref = 'no'
sc.tl.rank_genes_groups(subset,
    groupby=f'{set_name} identity',
    use_raw = True, ## raw contains log-transformed counts
    reference=ref,
    method='wilcoxon' 
    )

deg_res = sc.get.rank_genes_groups_df(subset,group=None)

name = set_name.replace(' ','_')
deg_res = deg_res[(deg_res['logfoldchanges'] >= 1) & (deg_res['pvals_adj'] < 0.05)].reset_index(drop=True)
deg_res.to_excel(f'{name}_top_1pct_degs_in_{ctype}.xlsx')

### Define T1RS epi and IL1b+ TAMs and investigate their connection in samples

In [ ]:
q_threshold = 0.95
score1_name = 'PDAC IL1B+ Macrophages'
score2_name = 'PDAC T1RS'

# Subset according to broad celltypes
subset1 = adata_ref[adata_ref.obs['refined_celltypes'] == 'Myeloid']
subset2 = adata_ref[adata_ref.obs['refined_celltypes'] == 'Epithelial']

# Find IL1b+ Macrophages in Myeloid & T1RS in Epithelial
subset1 = subset1[subset1.obs[score1_name] >= subset1.obs[score1_name].quantile(q_threshold)]
subset2 = subset2[subset2.obs[score2_name] >= subset2.obs[score2_name].quantile(q_threshold)]


In [ ]:
n_score1 = subset1.obs['sample'].value_counts()
n_score2 = subset2.obs['sample'].value_counts()

df = pd.merge(n_score1,n_score2,left_index=True,right_index=True,how='outer')
df.columns =[score1_name,score2_name]
df = df.fillna(0)

# Normalize with the total number of samples from each sample
df  = df.div(adata_ref.obs['sample'].value_counts()[df.index],axis=0)

pearsonr(df[score1_name],df[score2_name])
# PearsonRResult(statistic=0.08088847510906655, pvalue=0.4700385141957824)

# There is no correlation between the number of IL1b+ macrophages and T1RS epithelial cells
# Across the prostate cancer single-cell samples